In [1]:
from multiprocessing import Pool, current_process, Queue, Manager
from kdpipelines.Pipeline import Pipeline
from kdpipelines import Adapters, Processors, Sinks
from functools import partial
import json

/home/daniel/anaconda3/envs/KD/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
06/30/2021 12:32:43 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
[nltk_data] Downloading package punkt to /home/daniel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="aifb-ls3-vm1.aifb.kit.edu", username="", password="", index="document")
e_adapt = Adapters.ElasticsearchAdapter(document_store, batch_size=1000, filters={"_status":["1"]})
update_pipeline = Pipeline(e_adapt, batch_size=100, cpus=10, max_runtime=60)

06/30/2021 12:40:03 - INFO - elasticsearch -   HEAD http://aifb-ls3-vm1.aifb.kit.edu:9200/ [status:200 request:0.055s]
06/30/2021 12:40:03 - INFO - elasticsearch -   HEAD http://aifb-ls3-vm1.aifb.kit.edu:9200/document [status:200 request:0.018s]
06/30/2021 12:40:03 - INFO - elasticsearch -   GET http://aifb-ls3-vm1.aifb.kit.edu:9200/document [status:200 request:0.019s]
06/30/2021 12:40:03 - INFO - elasticsearch -   PUT http://aifb-ls3-vm1.aifb.kit.edu:9200/document/_mapping [status:200 request:0.025s]
06/30/2021 12:40:03 - INFO - elasticsearch -   HEAD http://aifb-ls3-vm1.aifb.kit.edu:9200/label [status:200 request:0.019s]
06/30/2021 12:40:03 - INFO - elasticsearch -   POST http://aifb-ls3-vm1.aifb.kit.edu:9200/document/_count [status:200 request:0.030s]
06/30/2021 12:40:04 - INFO - elasticsearch -   DELETE http://aifb-ls3-vm1.aifb.kit.edu:9200/_search/scroll [status:200 request:0.057s]


In [3]:
with open('../../data/pwc_metadata/datasets_preprocessed.json') as f:
    datasets = json.load(f)
update_pipeline.add_processor(Processors.StringMatchingProcessor(datasets, info_field_name='datasets', info_key='pwc_url'))

In [4]:
update_pipeline.add_processor(Processors.MetadataFieldAdder("_status", 1))

In [5]:
local_document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document", duplicate_documents='overwrite')
update_pipeline.add_sink(Sinks.ElasticsearchSink(local_document_store))

06/30/2021 12:32:48 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.004s]
06/30/2021 12:32:48 - INFO - elasticsearch -   HEAD http://localhost:9200/document [status:200 request:0.006s]
06/30/2021 12:32:48 - INFO - elasticsearch -   GET http://localhost:9200/document [status:200 request:0.004s]
06/30/2021 12:32:48 - INFO - elasticsearch -   PUT http://localhost:9200/document/_mapping [status:200 request:0.012s]
06/30/2021 12:32:48 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.003s]


In [6]:
print(update_pipeline)

(ElasticsearchAdapter) ---> *StringMatchingProcessor* ---> *MetadataFieldAdder*
===> |_ElasticsearchSink_|


In [7]:
update_pipeline.process()

06/30/2021 12:32:55 - INFO - root -   Pipeline processing started. A total of 6183181 documents will be processed.


06/30/2021 12:32:55 - INFO - elasticsearch -   POST http://aifb-ls3-vm1.aifb.kit.edu:9200/document/_search?scroll=1d&size=1000 [status:200 request:0.515s]
06/30/2021 12:33:31 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.943s]
06/30/2021 12:33:32 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.013s]
06/30/2021 12:33:33 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.011s]
06/30/2021 12:33:34 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.011s]
06/30/2021 12:33:35 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.009s]
06/30/2021 12:33:36 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.010s]
06/30/2021 12:33:37 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_fo

06/30/2021 12:33:41 - INFO - elasticsearch -   POST http://aifb-ls3-vm1.aifb.kit.edu:9200/_search/scroll [status:200 request:0.504s]
06/30/2021 12:34:16 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.216s]
06/30/2021 12:34:17 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.007s]
06/30/2021 12:34:18 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.005s]
06/30/2021 12:34:19 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.006s]
06/30/2021 12:34:21 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.006s]
06/30/2021 12:34:22 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.008s]
06/30/2021 12:34:23 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:

Processing batches (10 minibatches at once): [############################################################] 6184/6184 Processed documents: 2000 Output Documents: 2000


In [12]:
adapt = Adapters.ElasticsearchAdapter(local_document_store, batch_size=1000, filters={"has_datasets":["true"]})
len(adapt)

06/30/2021 12:36:07 - INFO - elasticsearch -   POST http://localhost:9200/document/_count [status:200 request:0.005s]
06/30/2021 12:36:07 - INFO - elasticsearch -   DELETE http://localhost:9200/_search/scroll [status:200 request:0.002s]


68

In [13]:
adapt.generate_documents(5)

06/30/2021 12:36:12 - INFO - elasticsearch -   POST http://localhost:9200/document/_search?scroll=1d&size=1000 [status:200 request:0.039s]


[{'text': 'While some studies emphasize the feasibility of using Wikipedia data to forecast influenza\xa0, dengue\xa0, zika\xa0, and other diseases\xa0, other research concludes behavioral heterogeneity on social platforms like Wikipedia cannot reliably beat canonical epidemiological forecasting methods\xa0, . We emphasize the goal of our study is not to evaluate the effectiveness of Wikipedia data in supporting emergency management or forecasting coronavirus incidence. Instead, we expect that coronavirus-related activity on Wikipedia will mirror previous disease outbreaks in terms of spikes of information seeking and contributions from editors of medical topics.',
  'score': None,
  'probability': None,
  'question': None,
  'meta': {'arixive-id': '2006.08899',
   'mag_id': 3035517000,
   '_split_id': 14,
   '_status': 1,
   'datasets': '[{"title": "dengue", "pwc_url": "https://paperswithcode.com/dataset/dengue", "span": [93, 99]}]',
   'has_datasets': 'true'},
  'embedding': None,
  

In [15]:
len(e_adapt)

241500

In [16]:
e_adapt.generate_documents(5)

06/30/2021 12:40:25 - INFO - elasticsearch -   POST http://aifb-ls3-vm1.aifb.kit.edu:9200/document/_search?scroll=1d&size=1000 [status:200 request:0.382s]


[{'text': 'These fine-grained results also provide users with a measure concerning the expected adoption rate of the simulated mobility initiative, so as to figure out beforehand if the initiative can potentially succeed or not. Technically, a Tangramob simulation requires four inputs: the urban road network of the area under study, a representative population of the area with the mobility agendas of people. the description of the mobility services already offered by the city: public transport timetable, etc.',
  'score': None,
  'probability': None,
  'question': None,
  'meta': {'arixive-id': '1805.10906',
   'mag_id': 2803352990,
   '_split_id': 15,
   '_status': 1,
   'Section': 'results'},
  'embedding': None,
  'id': '3f1f4fc1141c8c8f93d0b8ebc67578c2'},
 {'text': 'It is worth mentioning that the definition of an agent population is certainly the most complex and critical information to supply, in particular in relation to profiles, and details on daily travels. Obviously, the mor